# Rapid Intensification (RI) Prediction — Atlantic Hurricanes (HURDAT2)

**Goal:** Build a machine learning classifier that predicts whether a tropical cyclone will **rapidly intensify** within the next 24 hours, using only information available *up to the current observation time*.

**Definition (label):**
Rapid Intensification (RI) = **maximum sustained wind increases by ≥ 30 knots within 24 hours**.

**Dataset:** Kaggle — Atlantic Hurricane Dataset (HURDAT2), cleaned CSV format (`hurdat2.csv`).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path

from sklearn.model_selection import GroupShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    average_precision_score, confusion_matrix, classification_report
)

from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

DATA_PATH = Path("hurdat2.csv")  # put hurdat2.csv in same folder as notebook


In [ ]:
df = pd.read_csv(DATA_PATH)
print("Shape:", df.shape)
df.head()

## Dataset Overview

This dataset contains storm track observations (typically 6-hour intervals).
Each row is an observation for a storm at a timestamp, including:

- **Storm metadata:** `storm_id`, `storm_name`
- **Time:** `date`, `time`
- **Location:** `latitude`, `longitude`
- **Intensity:** `maximum_sustained_wind_knots` (target driver), and often pressure-related fields if present
- **Storm status:** `status_of_system` (e.g., TS, HU)

We will:
1. Clean and parse timestamps.
2. Convert latitude/longitude strings to numeric.
3. Create an RI label from wind change over the next 24 hours.
4. Train multiple ML models using proper storm-wise splits.

In [ ]:
df.info()

missing_pct = (df.isna().mean() * 100).sort_values(ascending=False)
missing_pct.head(20)

In [ ]:
def parse_lat(lat):
    # often like "18.0N" or "18.0"
    if pd.isna(lat):
        return np.nan
    s = str(lat).strip()
    if s[-1] in ["N", "S"]:
        val = float(s[:-1])
        return val if s[-1] == "N" else -val
    return float(s)

def parse_lon(lon):
    # often like "65.0W" or "65.0"
    if pd.isna(lon):
        return np.nan
    s = str(lon).strip()
    if s[-1] in ["E", "W"]:
        val = float(s[:-1])
        return val if s[-1] == "E" else -val
    return float(s)

# timestamp: date as YYYYMMDD and time as HHMM (or sometimes integer like 0, 600, 1200, 1800)
df["date"] = df["date"].astype(str)
df["time"] = df["time"].astype(str).str.zfill(4)

df["timestamp"] = pd.to_datetime(df["date"] + df["time"], format="%Y%m%d%H%M", errors="coerce", utc=True)

df["lat"] = df["latitude"].apply(parse_lat)
df["lon"] = df["longitude"].apply(parse_lon)

WIND_COL = "maximum_sustained_wind_knots"
df[WIND_COL] = pd.to_numeric(df[WIND_COL], errors="coerce").replace(-99, np.nan)

print(df[["storm_id","storm_name","timestamp","lat","lon",WIND_COL]].head())
print("Timestamp null %:", df["timestamp"].isna().mean()*100)

## Exploratory Data Analysis (EDA)

We will look at:
- distribution of wind speed
- storm status breakdown
- geographic scatter plot
- time coverage

In [ ]:
fig, ax = plt.subplots(figsize=(10,4))
df[WIND_COL].dropna().hist(bins=50, ax=ax)
ax.set_title("Maximum Sustained Wind (knots) distribution")
ax.set_xlabel("knots")
ax.set_ylabel("count")
plt.show()

df["status_of_system"].value_counts().head(15)

In [ ]:
sample = df.dropna(subset=["lat","lon"]).sample(n=min(5000, len(df)), random_state=RANDOM_SEED)

fig, ax = plt.subplots(figsize=(8,6))
ax.scatter(sample["lon"], sample["lat"], s=3)
ax.set_title("Track point locations (sample)")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
plt.show()

## Label Construction: Rapid Intensification (RI)

We create a binary label **RI_24h**:

- For each storm observation at time *t*, we find the wind at time *t + 24 hours* (or closest observation at that horizon).
- If `wind(t+24h) - wind(t) >= 30 knots`, then **RI_24h = 1**, else 0.

**Important:** Our features must use only data available at or before time *t* to avoid leakage.

In [ ]:
df = df.sort_values(["storm_id", "timestamp"]).reset_index(drop=True)

# We'll approximate "24h ahead" using 4 steps of 6-hourly data.
# But because some storms have irregularities, we do a safer merge using shift by 4 within each storm.
# This is a common and reasonable approach for HURDAT2 6-hourly tracks.

df["wind_t"] = df[WIND_COL]
df["wind_t_plus_24h"] = df.groupby("storm_id")["wind_t"].shift(-4)  # 4*6h = 24h
df["delta_wind_24h"] = df["wind_t_plus_24h"] - df["wind_t"]

df["RI_24h"] = (df["delta_wind_24h"] >= 30).astype(int)

# Drop rows where label can't be computed (end of storm) or wind missing
model_df = df.dropna(subset=["wind_t","wind_t_plus_24h","timestamp","lat","lon"]).copy()

print("Model rows:", model_df.shape)
print("RI positive rate:", model_df["RI_24h"].mean())
model_df[["storm_id","timestamp","wind_t","wind_t_plus_24h","delta_wind_24h","RI_24h"]].head(10)

## Feature Engineering

We will build features that mimic a real monitoring/forecasting setup:

**Current-state features**
- wind now, lat, lon, storm status

**Trend features (past-only)**
- wind change over last 6h, 12h, 24h (computed via lagged values within each storm)
- recent rolling mean

These are simple but strong baseline signals and align with the “textbook ML workflow” before adding deep learning.

In [ ]:
def add_lags(group):
    g = group.sort_values("timestamp").copy()
    g["wind_lag_1"] = g["wind_t"].shift(1)   # 6h ago
    g["wind_lag_2"] = g["wind_t"].shift(2)   # 12h ago
    g["wind_lag_4"] = g["wind_t"].shift(4)   # 24h ago

    g["dwind_6h"]  = g["wind_t"] - g["wind_lag_1"]
    g["dwind_12h"] = g["wind_t"] - g["wind_lag_2"]
    g["dwind_24h_past"] = g["wind_t"] - g["wind_lag_4"]

    g["wind_rollmean_4"] = g["wind_t"].rolling(4).mean()  # last 24h avg
    return g

feat_df = model_df.groupby("storm_id", group_keys=False).apply(add_lags)

# Drop rows where lag features unavailable
feat_df = feat_df.dropna(subset=["wind_lag_1","wind_lag_2","wind_lag_4","wind_rollmean_4"]).copy()

print("After lag features:", feat_df.shape)
feat_df[["storm_id","timestamp","wind_t","dwind_6h","dwind_12h","dwind_24h_past","wind_rollmean_4","RI_24h"]].head()

## Train / Validation / Test Split (Storm-wise)

We must split by **storm_id** so that observations from the same storm do not leak across train/val/test.

We will do:
- Train: 70%
- Validation: 15%
- Test: 15%

This matches realistic generalization: predicting on **new storms**.

In [ ]:
X = feat_df.copy()
y = feat_df["RI_24h"].astype(int).copy()
groups = feat_df["storm_id"].copy()

# First split: train vs temp
gss1 = GroupShuffleSplit(n_splits=1, test_size=0.30, random_state=RANDOM_SEED)
train_idx, temp_idx = next(gss1.split(X, y, groups=groups))

train = feat_df.iloc[train_idx].copy()
temp  = feat_df.iloc[temp_idx].copy()

# Second split: val vs test from temp
gss2 = GroupShuffleSplit(n_splits=1, test_size=0.50, random_state=RANDOM_SEED)
val_idx, test_idx = next(gss2.split(temp, temp["RI_24h"], groups=temp["storm_id"]))

val  = temp.iloc[val_idx].copy()
test = temp.iloc[test_idx].copy()

print("Train:", train.shape, "Val:", val.shape, "Test:", test.shape)
print("Train RI rate:", train["RI_24h"].mean())
print("Val   RI rate:", val["RI_24h"].mean())
print("Test  RI rate:", test["RI_24h"].mean())

## Modeling with Pipelines

We will use scikit-learn Pipelines to ensure clean, reproducible preprocessing:
- numeric: impute + scale
- categorical: impute + one-hot encode

Then we train multiple models:
- Perceptron
- Logistic Regression
- SVM (linear and RBF)
- KNN
- Decision Tree
- Random Forest

We select the best model based on validation metrics.
Because RI is imbalanced, we will track:
- Accuracy (easy to inflate)
- F1 (balance precision/recall)
- ROC-AUC
- PR-AUC (Average Precision) — best for imbalanced positives

In [ ]:
target = "RI_24h"

feature_cols_num = [
    "wind_t", "lat", "lon",
    "wind_lag_1", "wind_lag_2", "wind_lag_4",
    "dwind_6h", "dwind_12h", "dwind_24h_past",
    "wind_rollmean_4"
]

feature_cols_cat = ["status_of_system"]

# Some datasets might have status missing; keep safe
for c in feature_cols_cat:
    if c not in feat_df.columns:
        feat_df[c] = np.nan
        train[c] = np.nan
        val[c] = np.nan
        test[c] = np.nan

X_train, y_train = train[feature_cols_num + feature_cols_cat], train[target]
X_val, y_val     = val[feature_cols_num + feature_cols_cat], val[target]
X_test, y_test   = test[feature_cols_num + feature_cols_cat], test[target]

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, feature_cols_num),
        ("cat", categorical_transformer, feature_cols_cat),
    ]
)

In [ ]:
def eval_model(name, model, Xtr, ytr, Xva, yva):
    pipe = Pipeline(steps=[("preprocess", preprocess), ("model", model)])
    pipe.fit(Xtr, ytr)

    pred = pipe.predict(Xva)

    # probabilities/scores for AUC
    if hasattr(pipe.named_steps["model"], "predict_proba"):
        proba = pipe.predict_proba(Xva)[:, 1]
    elif hasattr(pipe.named_steps["model"], "decision_function"):
        scores = pipe.decision_function(Xva)
        # convert scores to 0..1-ish via minmax for PR-AUC stability
        proba = (scores - scores.min()) / (scores.max() - scores.min() + 1e-9)
    else:
        proba = None

    acc = accuracy_score(yva, pred)
    prec = precision_score(yva, pred, zero_division=0)
    rec = recall_score(yva, pred, zero_division=0)
    f1 = f1_score(yva, pred, zero_division=0)

    roc = roc_auc_score(yva, proba) if proba is not None else np.nan
    pr  = average_precision_score(yva, proba) if proba is not None else np.nan

    return {
        "model": name,
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "roc_auc": roc,
        "pr_auc": pr,
        "pipeline": pipe
    }

In [ ]:
models = [
    ("Perceptron", Perceptron(random_state=RANDOM_SEED)),
    ("LogReg", LogisticRegression(max_iter=2000, class_weight="balanced", random_state=RANDOM_SEED)),
    ("LinearSVM", SVC(kernel="linear", class_weight="balanced", probability=True, random_state=RANDOM_SEED)),
    ("RBFSVM", SVC(kernel="rbf", class_weight="balanced", probability=True, random_state=RANDOM_SEED)),
    ("KNN", KNeighborsClassifier(n_neighbors=15)),
    ("DecisionTree", DecisionTreeClassifier(max_depth=6, random_state=RANDOM_SEED, class_weight="balanced")),
    ("RandomForest", RandomForestClassifier(
        n_estimators=300, random_state=RANDOM_SEED, class_weight="balanced", n_jobs=-1
    )),
]

results = []
for name, m in models:
    out = eval_model(name, m, X_train, y_train, X_val, y_val)
    results.append(out)

score_table = pd.DataFrame([{k:v for k,v in r.items() if k!="pipeline"} for r in results])
score_table.sort_values("pr_auc", ascending=False)

## Model Selection

We choose the best model based primarily on **Validation PR-AUC**, with F1 as a secondary metric.
PR-AUC is ideal because RI is relatively rare (imbalanced classification).

In [ ]:
best = sorted(results, key=lambda r: (r["pr_auc"], r["f1"]), reverse=True)[0]
best_name = best["model"]
best_pipe = best["pipeline"]

print("Best model:", best_name)
print(score_table.sort_values("pr_auc", ascending=False).head(10))

# Evaluate on TEST
test_pred = best_pipe.predict(X_test)

if hasattr(best_pipe.named_steps["model"], "predict_proba"):
    test_proba = best_pipe.predict_proba(X_test)[:, 1]
elif hasattr(best_pipe.named_steps["model"], "decision_function"):
    scores = best_pipe.decision_function(X_test)
    test_proba = (scores - scores.min()) / (scores.max() - scores.min() + 1e-9)
else:
    test_proba = None

print("\nTEST METRICS")
print("Accuracy:", accuracy_score(y_test, test_pred))
print("Precision:", precision_score(y_test, test_pred, zero_division=0))
print("Recall:", recall_score(y_test, test_pred, zero_division=0))
print("F1:", f1_score(y_test, test_pred, zero_division=0))
print("ROC-AUC:", roc_auc_score(y_test, test_proba) if test_proba is not None else "N/A")
print("PR-AUC:", average_precision_score(y_test, test_proba) if test_proba is not None else "N/A")

print("\nConfusion matrix:\n", confusion_matrix(y_test, test_pred))
print("\nClassification report:\n", classification_report(y_test, test_pred, zero_division=0))

In [ ]:
# Optional: simple feature importance if RandomForest was best
if best_name == "RandomForest":
    rf = best_pipe.named_steps["model"]
    # After preprocessing, feature names expand due to one-hot.
    ohe = best_pipe.named_steps["preprocess"].named_transformers_["cat"].named_steps["onehot"]
    cat_names = ohe.get_feature_names_out(feature_cols_cat)
    all_names = feature_cols_num + list(cat_names)

    importances = pd.Series(rf.feature_importances_, index=all_names).sort_values(ascending=False)
    display(importances.head(20))

    fig, ax = plt.subplots(figsize=(10,5))
    importances.head(15).sort_values().plot(kind="barh", ax=ax)
    ax.set_title("Top feature importances (RandomForest)")
    plt.show()

## Conclusions & NASA DEVELOP Framing

### What we built
A reproducible ML pipeline that predicts **Rapid Intensification (RI)** within the next 24 hours from storm track observations.

### What the results mean
- Strong validation/test performance suggests the model captures important short-term intensification signals such as:
  - current wind intensity
  - recent wind trends (6h/12h/24h)
  - storm location and system status

### Why this matters (hazards / applications)
Rapid intensification is operationally challenging and can reduce warning lead time.
A lightweight ML classifier can support:
- early alerts for potential RI episodes
- prioritizing storms for deeper dynamical analysis
- decision support for hazard response workflows

### Limitations
- Label is derived only from wind and track timing; no ocean heat content, shear, SST, satellite imagery, etc.
- Dataset is limited to Atlantic best-track; generalization to other basins requires global datasets (IBTrACS).
- 24h-ahead “shift -4” assumes regular 6-hour cadence (reasonable for HURDAT2 but still an approximation).

### Next Steps (to make this NASA-grade)
1. Extend to **IBTrACS** global best-track.
2. Add environmental predictors (SST, vertical wind shear, reanalysis).
3. Add spatial features (distance to land, coast proximity).
4. Use time-series models or gradient boosting with careful leakage control.